In [1]:
import pandas as pd
# from nlpia.data.loaders import read_csv

df = pd.read_csv('../shared-resources/chatbot/testset.csv')
df.head()

,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,anyone knows why my stock oneiric exports env ...,nice thanks! __eou__,"wrong channel for it, but check efnet.org, uno...","every time the kernel changes, you will lose v...",ok __eou__,!nomodeset > acer __eou__ I'm assuming it is a...,http://www.ubuntu.com/project/about-ubuntu/der...,thx __eou__ unfortunately the program isn't in...,how can I check? By doing a recovery for testi...,my humble apologies __eou__,#ubuntu-offtopic __eou__
1,i set up my hd such that i have to type a pass...,"so you dont know, ok, anyone else? __eou__ you...","nmap is nice, but it wasn't what I was looking...",ok __eou__,cdrom worked fine on windows. __eou__ i dont ...,"ah yes, i have read return as rerun __eou__",hm? __eou__,"not the case, LTS is every other .04 release. ...",Pretty much __eou__,I used the one I downloaded from AMD __eou__,"ffmpeg is part of the package , quixotedon , a..."
2,im trying to use ubuntu on my macbook pro reti...,just wondering how it runs __eou__,"yes, that's what I did, exported it to a ""id_d...",nothing - i am talking about the question of m...,that should fix the fonts being too large __eou__,"okay, so hcitool echos back hci0 <mac address ...",I get to the menu with options such as 'try ub...,why do u need analyzer __eou__ it is a toy __e...,Cntrl-C may stop the command but it doesn't fi...,"if you're only going to run Ubuntu, just get a...",the ones which are not picked up at the moment...
3,no suggestions? __eou__ links? __eou__ how can...,you cant load anything via usb or cd when luks...,-p sorry... __eou__ nmap -p22 __eou__ It d...,i guess so i can't even launch it. __eou__,noted __eou__,rxvt-unicode is one __eou__,I tarred all of ~ __eou__,I tarred all of ~ __eou__,"I don't really know if I can help, but I was c...","that works just fine, thanks! __eou__",thank you __eou__
4,I just added a second usb printer but not sure...,i was setting it up under the printer configur...,i'd say the most commonly venue would be via L...,"the old hardy man page, http://manpages.ubuntu...",i'll give a try __eou__,"by the way, the url you posted for davfs is fr...",http://ubuntuforums.org/showthread.php?t=15498...,"So I load up putty gui, then what do I do? __e...","you should read error messages, it says 'are ...",waiting the college semester to close just to ...,I was calling myself a jerk. All I know is tha...


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [28]:
TfidfVectorizer??

In [15]:
vectors = tfidf.fit_transform(['the hello world', 'another time'])
vectors

<2x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [16]:
vectors = vectors.todense()
vectors

matrix([[ 0.        ,  0.57735027,  0.57735027,  0.        ,  0.57735027],
        [ 0.70710678,  0.        ,  0.        ,  0.70710678,  0.        ]])

In [9]:
tfidf.vocabulary_

{'another': 0, 'hello': 1, 'time': 2, 'world': 3}

In [21]:
tfidf = TfidfVectorizer(min_df=8, max_df=.3, max_features=100000)
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.3, max_features=100000, min_df=8,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [38]:
tfidf = tfidf.fit(pd.concat([df[df.columns[i]] for i in range(11)]))

In [33]:
list(tfidf.vocabulary_)[:10]

['anyone',
 'knows',
 'why',
 'my',
 'stock',
 'oneiric',
 'exports',
 'env',
 'var',
 'username']

In [40]:
X = tfidf.transform(df.Context)
X = X.todense()

In [41]:
y = tfidf.transform(df['Ground Truth Utterance']).todense()

In [42]:
from sklearn.metrics.pairwise import cosine_distances

In [43]:
# for string
def get_statement(s='Hi'):
    q = tfidf.transform([s]).todense()[0]
    similarity = 0
    best = 0
    for i, v in enumerate(X):
        # print(i, q, v)
        sim = 2 - cosine_distances(q, v)
        if sim > similarity:
            similarity = sim
            best = i
    return df.Context.iloc[best], best

In [50]:
get_statement(df.Context.iloc[0])[1]

0

In [51]:
def reply(s):
    return df['Ground Truth Utterance'].iloc[get_statement(s)[1]]

In [52]:
reply(df.Context.iloc[0])

'nice thanks! __eou__'

In [81]:
from sklearn.decomposition import PCA
pca100 = PCA(n_components=100)
pca100 = pca100.fit(tfidf.transform(df.Context).todense())

In [82]:
# X is context only
X_100d = pca100.transform(X)

In [83]:
s=df.Context.iloc[1]
pca100.transform(tfidf.transform([s]).todense())[0]

array([-0.13638994,  0.07041551, -0.05269807,  0.03593849,  0.09723707,
        0.04843653, -0.04354091, -0.11602665,  0.02806283, -0.03740251,
        0.04947289,  0.00021808, -0.0033221 ,  0.03041059, -0.01776948,
       -0.02973298,  0.05986818,  0.04139537, -0.02262693, -0.04556857,
       -0.07378996, -0.03999293, -0.04436306, -0.06444699, -0.11689505,
       -0.02564208, -0.08167965,  0.05604148, -0.02952699, -0.03491643,
       -0.00665106, -0.00821851,  0.01411447,  0.02163809,  0.04303451,
        0.03741294,  0.07120954, -0.01617283,  0.05946246, -0.01257618,
        0.00884914, -0.01447142, -0.00944132,  0.00934043,  0.00460624,
       -0.00148847, -0.07815515,  0.08887428,  0.00194284, -0.02763396,
        0.04351698, -0.08278025, -0.02743409, -0.00991519,  0.01416716,
        0.06920658, -0.00963584,  0.00055206,  0.06052855, -0.01548511,
        0.04482815,  0.1089491 , -0.03999688,  0.06755321, -0.00733648,
       -0.01542249, -0.02347796, -0.03453771,  0.01220991,  0.04

In [96]:
# for pca vector, return the closest ground truth
def get_pca_statement(s, X, pca):
    q = pca.transform(tfidf.transform([s]).todense())[0]
    similarity = 0
    best = 0
    for i, v in enumerate(X):
        # print(i, q, v)
        sim = 2 - cosine_distances(pd.np.array([q]), pd.np.array([v]))
        if sim > similarity:
            similarity = sim
            best = i
    return df.Context.iloc[best], best

In [101]:
get_pca_statement('I set up my hd and now I have to type a passphrase to access it at start. how can I remove that password, and just boot it up normally. I am tired of having reboots where i need to be at terminal to type passwd in.', X_100d, pca100)

('i set up my hd such that i have to type a passphrase to access it at boot. how can i remove that passwrd, and just boot up normal. i did this at install, it works fine, just tired of having reboots where i need to be at terminal to type passwd in. help? __eou__ __eot__ backup your data, and re-install without encryption "might" be the easiest method __eou__ __eot__ ',
 1)

In [99]:
s

'i set up my hd such that i have to type a passphrase to access it at boot. how can i remove that passwrd, and just boot up normal. i did this at install, it works fine, just tired of having reboots where i need to be at terminal to type passwd in. help? __eou__ __eot__ backup your data, and re-install without encryption "might" be the easiest method __eou__ __eot__ '

In [86]:
pca50 = PCA(n_components=50)
pca50 = pca50.fit(tfidf.transform(df.Context).todense())
X_50d = pca50.transform(X)

In [87]:
get_pca_statement(s, X_50d, pca50)

('so you dont know, ok, anyone else? __eou__ you are like, yah my mouse doesnt work, reinstall your os lolol what a joke __eou__',
 1)

In [91]:
pca10 = PCA(n_components=10)
pca10 = pca10.fit(tfidf.transform(df.Context).todense())
X_10d = pca10.transform(X)

In [92]:
get_pca_statement(s, X_10d, pca10)

('so you dont know, ok, anyone else? __eou__ you are like, yah my mouse doesnt work, reinstall your os lolol what a joke __eou__',
 1)

In [94]:
pca1 = PCA(n_components=1)
pca1 = pca1.fit(tfidf.transform(df.Context).todense())
X_1d = pca1.transform(X)

In [95]:
get_pca_statement(s, X_1d, pca1)

('so you dont know, ok, anyone else? __eou__ you are like, yah my mouse doesnt work, reinstall your os lolol what a joke __eou__',
 1)